In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [48]:
class auto_validating:
    def __init__(self, estimator, missing_values_in=[]):
        self.estimator = estimator
        self.missing_values_in = missing_values_in
        
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, traceback):
        pass
    
    def fit(self, data, labels):
        print("intercepting training...")
        
        # TODO We need to train a validation model for missing values here
        
        model = self.estimator.fit(data, labels)    
        return PredictionInterceptor(model)
        
    
class PredictionInterceptor:
    def __init__(self, transformer):
        self.transformer = transformer
        
    def score(self, data, labels):
        print("Intercepting prediction...")
        
        # TODO We need to apply the validation model here
        
        return self.transformer.score(data, labels)
    

In [49]:
data = pd.read_csv('../resources/data/adult/adult.csv')

train_data, test_data = train_test_split(data)
y_train = np.array(train_data['class'] == '>50K')
y_test = np.array(test_data['class'] == '>50K')

    
feature_transformation = ColumnTransformer(transformers=[
    ('categorical', OneHotEncoder(handle_unknown='ignore'), ['workclass', 'occupation', 'marital_status', 'education']),
    ('numeric', StandardScaler(), ['hours_per_week', 'age'])
])

pipeline = Pipeline([
    ('features', feature_transformation),
    ('learner', SGDClassifier(loss='log'))
])

with auto_validating(pipeline, missing_values_in=['education']) as validatable_pipeline:
    model = validatable_pipeline.fit(train_data, y_train)

model.score(test_data, y_test)


intercepting training...
Intercepting prediction...


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


0.8295049748188184